# MINDS JSON-LD writer
This script produces a collection of MINDS JSONs, which describe the dataset according to HBP minimal metadata criteria (MINDS v1).   
For more information please visit:   
https://github.com/HumanBrainProject/openMINDS

In [ ]:
# import required packages
import os
import pandas as pd
from braceexpand import braceexpand
import json

## Read study participant information from BIDS participants.tsv
Alternatively, participant information can be manually loaded into the 'participants_tsv' variable.

In [ ]:
# specify path to BIDS dataset
BIDSroot = "/path/to/BIDS"
# Read participants.tsv
participants_tsv = pd.read_csv(BIDSroot+"/participants.tsv", sep='\t',dtype={'participant_id': str})

# specify path where output MINDS JSON-LD schema shall be stored
MINDSroot = "/path/to/output"

## Generate output directory structure

In [ ]:
for x in list(braceexpand(MINDSroot+'/core/{dataset,specimengroup}/v1.0.0')): os.makedirs(x, exist_ok=True)
os.makedirs(MINDSroot+'/experiment/subject/v1.0.0', exist_ok=True)

## Define helper functions that write JSONs from schemas

The function *openMINDSschemaWriter* accepts arguments for schema type, etc.
The class *baseSchema* defines attributes common to all schemas. It also defines a default function for naming & writing JSON files (files are named: schemaName-0X.json)

All further classes are based on the *baseSchema* class. They define additional attributes (fill the keys of the specific schemas (JSON templates)). Additionally, in some cases, they may implement a modified function to rename JSONs; JSONs will be named according to the schema's *name* field.

Note: (un)comment lines related to the key-value pairs that should (not) appear in the resulting JSON files and modify fields according to the features your data set.

In [ ]:
def openMINDSschemaWriter(schema_type, MINDSroot, schema_dir, schema_name, schema_json, dataRoot, relatedIRI=""):
    if schema_type == "person":
        return personSchema(schema_type, MINDSroot, schema_dir, schema_name, schema_json, dataRoot, relatedIRI)
    if schema_type == "sex":
        return sexSchema(schema_type, MINDSroot, schema_dir, schema_name, schema_json, dataRoot, relatedIRI)
    if schema_type == "species":
        return speciesSchema(schema_type, MINDSroot, schema_dir, schema_name, schema_json, dataRoot, relatedIRI)
    if schema_type == "authority":
        return authoritySchema(schema_type, MINDSroot, schema_dir, schema_name, schema_json, dataRoot, relatedIRI)
    if schema_type == "approval":
        return approvalSchema(schema_type, MINDSroot, schema_dir, schema_name, schema_json, dataRoot, relatedIRI)
    if schema_type == "method":
        return methodsSchema(schema_type, MINDSroot, schema_dir, schema_name, schema_json, dataRoot, relatedIRI)
    if schema_type == "preparation":
        return preparationSchema(schema_type, MINDSroot, schema_dir, schema_name, schema_json, dataRoot, relatedIRI)
    if schema_type == "activity":
        return activitySchema(schema_type, MINDSroot, schema_dir, schema_name, schema_json, dataRoot, relatedIRI)
    if schema_type == "agecategory":
        return agecategorySchema(schema_type, MINDSroot, schema_dir, schema_name, schema_json, dataRoot, relatedIRI)
    if schema_type == "specimengroup":
        return specimengroupSchema(schema_type, MINDSroot, schema_dir, schema_name, schema_json, dataRoot, relatedIRI)
    if schema_type == "simulator":
        return simulatorSchema(schema_type, MINDSroot, schema_dir, schema_name, schema_json, dataRoot, relatedIRI)
    if schema_type == "subject":
        return subjectSchema(schema_type, MINDSroot, schema_dir, schema_name, schema_json, dataRoot, relatedIRI)
    if schema_type == "dataset":
        return datasetSchema(schema_type, MINDSroot, schema_dir, schema_name, schema_json, dataRoot, relatedIRI)

class baseSchema:
    def __init__(self, schema_type, MINDSroot, schema_dir, schema_name, schema_json, dataRoot, relatedIRI):
        self.schema_type = schema_type
        self.schema_dir = MINDSroot + schema_dir
        self.schema_name = schema_name
        self.schema_json = schema_json
        self.relatedIRI = relatedIRI
        self.fileNum = str(len([file for file in os.listdir(self.schema_dir) if os.path.isfile(os.path.join(self.schema_dir, file))]) + 1).zfill(3)
    def writeJSON(self):
        #generic JSON naming scheme [<schemaName>-0X.json]. override in specific class if "name" naming required.
        with open(self.schema_dir+"/"+self.schema_type+"-"+self.fileNum+".json","w") as f:
            json.dump(self.schema_json, f, indent=4)

class personSchema(baseSchema):
    def fillJSON(self):
        self.schema_json["@type"] = "https://schema.hbp.eu/minds/"+self.schema_type+".schema.json"
        self.schema_json["@id"] = "minds"+ self.schema_dir.split(MINDSroot,1)[1] + "/" + self.schema_type + "-" + self.fileNum + ".json"
        self.schema_json["name"] = self.schema_name
        shortName = ""
        shortName = shortName + self.schema_name.split(",")[0] + ", "
        for w in self.schema_name.split(",")[1].split():
            shortName = shortName + w[0].upper() + ". "
        shortName.rstrip()
        self.schema_json["shortName"] = shortName

class sexSchema(baseSchema):
    def fillJSON(self):
        self.schema_json["@type"] = "https://schema.hbp.eu/minds/"+self.schema_type+".schema.json"
        self.schema_json["@id"] = "minds"+ self.schema_dir.split(MINDSroot,1)[1] + "/" + self.schema_type + "-" + self.fileNum + ".json"
        self.schema_json["name"] = self.schema_name
    def writeJSON(self): #overwrite baseSchema "writeJSON". #name.
        with open(self.schema_dir+"/"+self.schema_name+".json","w") as f:
            json.dump(self.schema_json, f, indent=4)

class speciesSchema(baseSchema):
    def fillJSON(self):
        self.schema_json["@type"] = "https://schema.hbp.eu/minds/"+self.schema_type+".schema.json"
        self.schema_json["@id"] = "minds"+ self.schema_dir.split(MINDSroot,1)[1] + "/" + self.schema_type + "-" + self.fileNum + ".json"
        self.schema_json["name"] = self.schema_name
        self.schema_json["ontologicalTerm"] = [{"@id": "ontologies/core/metazoa/v1.0.0/63b90ba0-66be-4969-8a6a-d19ebea01115"}]
    def writeJSON(self): #overwrite baseSchema "writeJSON". #name.
        with open(self.schema_dir+"/homo-sapiens.json","w") as f:
            json.dump(self.schema_json, f, indent=4)

class authoritySchema(baseSchema):
    def fillJSON(self):
        self.schema_json["@type"] = "https://schema.hbp.eu/minds/"+self.schema_type+".schema.json"
        self.schema_json["@id"] = "minds"+ self.schema_dir.split(MINDSroot,1)[1] + "/" + self.schema_type + "-" + self.fileNum + ".json"
        self.schema_json["name"] = self.schema_name

class approvalSchema(baseSchema):
    def fillJSON(self):
        self.schema_json["@type"] = "https://schema.hbp.eu/minds/"+self.schema_type+".schema.json"
        self.schema_json["@id"] = "minds"+ self.schema_dir.split(MINDSroot,1)[1] + "/" + self.schema_type + "-" + self.fileNum + ".json"
        self.schema_json["name"] = self.schema_name
        self.schema_json["generatedBy"] = [{"@id": "minds/ethics/authority/v1.0.0/"+file} for file in os.listdir(MINDSroot + "/ethics/authority/v1.0.0")]

class methodsSchema(baseSchema):
    def fillJSON(self):
        self.schema_json["@type"] = "https://schema.hbp.eu/minds/"+self.schema_type+".schema.json"
        self.schema_json["@id"] = "minds"+ self.schema_dir.split(MINDSroot,1)[1] + "/" + self.schema_type + "-" + self.fileNum + ".json"
        self.schema_json["name"] = self.schema_name
        self.schema_json["relatedIRI"] = self.relatedIRI

class preparationSchema(baseSchema):
    def fillJSON(self):
        self.schema_json["@type"] = "https://schema.hbp.eu/minds/"+self.schema_type+".schema.json"
        self.schema_json["@id"] = "minds"+ self.schema_dir.split(MINDSroot,1)[1] + "/" + self.schema_type + "-" + self.fileNum + ".json"
        self.schema_json["name"] = self.schema_name

class activitySchema(baseSchema):
    def fillJSON(self):
        self.schema_json["@type"] = "https://schema.hbp.eu/minds/"+self.schema_type+".schema.json"
        self.schema_json["@id"] = "minds"+ self.schema_dir.split(MINDSroot,1)[1] + "/" + self.schema_type + "-" + self.fileNum + ".json"
        self.schema_json["name"] = self.schema_name
        self.schema_json["approval"] = [{"@id": "minds/ethics/approval/v1.0.0/"+file} for file in os.listdir(MINDSroot + "/ethics/approval/v1.0.0")]
        self.schema_json["authority"] = [{"@id": "minds/ethics/authority/v1.0.0/"+file} for file in os.listdir(MINDSroot + "/ethics/authority/v1.0.0")]
        if self.schema_name == "MRI-T1w":
            self.schema_json["methods"] = [{"@id": "minds/experiment/method/v1.0.0/method-001.json"}]
        elif self.schema_name == "MRI-T2W":
            self.schema_json["methods"] = [{"@id": "minds/experiment/method/v1.0.0/method-002.json"}]
        elif self.schema_name == "MRI-T2STAR":
            self.schema_json["methods"] = [{"@id": "minds/experiment/method/v1.0.0/method-003.json"}]
        elif self.schema_name == "MRI-FLAIR":
            self.schema_json["methods"] = [{"@id": "minds/experiment/method/v1.0.0/method-004.json"}]
        elif self.schema_name == "resting state fMRI":
            self.schema_json["methods"] = [{"@id": "minds/experiment/method/v1.0.0/method-005.json"},
                                           {"@id": "minds/experiment/method/v1.0.0/method-08.json"}]
        elif self.schema_name == "DWI":
            self.schema_json["methods"] = [{"@id": "minds/experiment/method/v1.0.0/method-006.json"}]
        elif self.schema_name == "PET":
            self.schema_json["methods"] = [{"@id": "minds/experiment/method/v1.0.0/method-007.json"}]

        elif self.schema_name == "DWI-ImageProcessing": #(method#s: 6 & 9-15)
            methodNums = [str(6).zfill(3)] \
                        + [str(i).zfill(3) for i in range(9,15+1)]
            self.schema_json["methods"] = [{"@id": "minds/experiment/method/v1.0.0/method-{}.json".format(i)} for i in methodNums]

        elif self.schema_name == "T1-imageProcessing": #(method#s: 1 & 16-29)
            methodNums = [str(1).zfill(3)] \
                        + [str(i).zfill(3) for i in range(16,29+1)]
            self.schema_json["methods"] = [{"@id": "minds/experiment/method/v1.0.0/method-{}.json".format(i)} for i in methodNums]

        elif self.schema_name == "rsfMRI-ImageProcessing": #(method#s: 5 & 8 & 30-34 & 20 & 35 & 19 & 36-40)
            methodNums = [str(i).zfill(3) for i in [5,8]] \
                        + [str(i).zfill(3) for i in range(30,34+1)] \
                        + [str(20).zfill(3)] \
                        + [str(35).zfill(3)] \
                        + [str(19).zfill(3)] \
                        + [str(i).zfill(3) for i in range(36,40+1)]
            self.schema_json["methods"] = [{"@id": "minds/experiment/method/v1.0.0/method-{}.json".format(i)} for i in methodNums]

        elif self.schema_name == "PET-ImageProcessing": #(method#s: 7 & 41-44 & 34 & 22)
            methodNums = [str(7).zfill(3)] \
                        + [str(i).zfill(3) for i in range(41,44+1)] \
                        + [str(34).zfill(3)] \
                        + [str(22).zfill(3)]
            self.schema_json["methods"] = [{"@id": "minds/experiment/method/v1.0.0/method-{}.json".format(i)} for i in methodNums]

        elif self.schema_name == "create cortical surface and region mapping": #(method#s: 45-51)
            methodNums = [str(i).zfill(3) for i in range(45,51+1)]
            self.schema_json["methods"] = [{"@id": "minds/experiment/method/v1.0.0/method-{}.json".format(i)} for i in methodNums]

        elif self.schema_name == "compute source space":
            methodNums = [str(i).zfill(3) for i in range(52,54+1)] #(method#s: 52-54)
            self.schema_json["methods"] = [{"@id": "minds/experiment/method/v1.0.0/method-{}.json".format(i)} for i in methodNums]

        elif self.schema_name == "compute BEM model & EEG locations": #(method#s: 55-60)
            methodNums = [str(i).zfill(3) for i in range(55,60+1)]
            self.schema_json["methods"] = [{"@id": "minds/experiment/method/v1.0.0/method-{}.json".format(i)} for i in methodNums]

        elif self.schema_name == "compute forward solution": #(method#s: 61-63)
            methodNums = [str(i).zfill(3) for i in range(61,63+1)]
            self.schema_json["methods"] = [{"@id": "minds/experiment/method/v1.0.0/method-{}.json".format(i)} for i in methodNums]

        elif self.schema_name == "save derivatives accoording to TVB specifications": # (method#: 64-70)
            methodNums = [str(i).zfill(3) for i in range(64,70+1)]
            self.schema_json["methods"] = [{"@id": "minds/experiment/method/v1.0.0/method-{}.json".format(i)} for i in methodNums]

        elif self.schema_name == "PhenotypicandAssessmentData": #(method#s: 71-81)
            methodNums = [str(i).zfill(3) for i in range(71,81+1)]
            self.schema_json["methods"] = [{"@id": "minds/experiment/method/v1.0.0/method-{}.json".format(i)} for i in methodNums]

        self.schema_json["preparation"] = [{"@id": "minds/core/preparation/v1.0.0/"+file} for file in os.listdir(MINDSroot + "/core/preparation/v1.0.0")]

class agecategorySchema(baseSchema):
    def fillJSON(self):
        self.schema_json["@type"] = "https://schema.hbp.eu/minds/"+self.schema_type+".schema.json"
        self.schema_json["@id"] = "minds"+ self.schema_dir.split(MINDSroot,1)[1] + "/" + self.schema_type + "-" + self.fileNum + ".json"
        self.schema_json["name"] = self.schema_name

class specimengroupSchema(baseSchema):
    def fillJSON(self):
        self.schema_json["@type"] = "https://schema.hbp.eu/minds/"+self.schema_type+".schema.json"
        self.schema_json["@id"] = "minds"+ self.schema_dir.split(MINDSroot,1)[1] + "/" + self.schema_type + "-" + self.fileNum + ".json"
        self.schema_json["name"] = self.schema_name
        self.schema_json["subjects"] = []


class subjectSchema(baseSchema):
    def fillJSON(self):
        self.schema_json["@type"] = "https://schema.hbp.eu/minds/"+self.schema_type+".schema.json"
        self.schema_json["@id"] = "minds"+ self.schema_dir.split(MINDSroot,1)[1] + "/" + self.schema_name + ".json"
        self.schema_json["name"] = "HCP_sub-" + self.schema_name
        participants_tsv = pd.read_csv(BIDSroot + "/participants.tsv", sep='\t',dtype={'participant_id': str})
        self.schema_json["age"] = str(participants_tsv[participants_tsv["participant_id"]==self.schema_name]["Age"].item()) + " years"
        self.schema_json["ageCategory"] = [{"@id" : "minds/core/agecategory/v1.0.0/97eaac65-ec72-4435-9e20-5b2308076060"}] #if participants_tsv[participants_tsv["participant_id"]==self.schema_name]["Age"].item() > 18 else []
        self.schema_json["sex"] = [{"@id" : "minds/core/sex/v1.0.0/85ba5561-e9bf-41a8-b0fe-bd17ae0f7b2c"}] if participants_tsv[participants_tsv["participant_id"]==self.schema_name]["Sex"].item() == "M" else [{"@id" : "minds/core/sex/v1.0.0/f7c82ad1-b4ba-495a-ba81-145d8dbf82ea"}]
        self.schema_json["species"] = [{"@id" : "minds/core/species/v1.0.0/0ea4e6ba-2681-4f7d-9fa9-49b915caaac9"}]
    def writeJSON(self): #overwrite baseSchema "writeJSON". #name by subject ID.
        with open(self.schema_dir+"/"+"HCP_sub-"+self.schema_name.replace(" ","")+".json","w") as f:
            json.dump(self.schema_json, f, indent=4)

class datasetSchema(baseSchema):
    def fillJSON(self):
        self.schema_json["@type"] = "https://schema.hbp.eu/minds/"+self.schema_type+".schema.json"
        self.schema_json["@id"] = "minds"+ self.schema_dir.split(MINDSroot,1)[1] + "/" + self.schema_type + "-" + self.fileNum + ".json"
        self.schema_json["name"] = self.schema_name
        self.schema_json["description"] = "This data set contains structural and functional connectivity and region-average fMRI time series from 785 participants of the Human Connectome Project (HCP) S900 data set. HCP builds network maps (connectomes) of the human brain and provides data for research into brain disorders, but also seeks to understand healthy brains and developmental processes. HCP uses non-invasive imaging technologies: resting-state fMRI, task-based fMRI, MEG, diffusion MRI and also performs behavioral and genetic testing. Data was acquired using customized sequences with high spatial and temporal resolution: 2 mm isotropic voxel size and a TR of 720 ms for fMRI and 1.25 mm isotropic voxel size and a diffusion weighting of up to b = 3000 s/mm2 and >=128 unique directions for dwMRI. HCP's minimal preprocessing pipelines were used for preprocessing, involving distortion removal, surface mapping, registration, and alignment to standard space. Correction of EPI and eddy-current distortions uses phase-encoding direction-reversed images for each diffusion direction. Region-parcellation is based on HCP's Glasser/MMP1 atlas. Full-brain tractography with 25 Million produced tracks yielded structural connectomes.",
#        self.schema_json["owners"] = [{"@id": "minds/core/person/v1.0.0/"+os.listdir(MINDSroot + "/core/person/v1.0.0")[0]}],
#        self.schema_json["contributors"] = [ {"@id": "minds/core/person/v1.0.0/"+file} for file in os.listdir(MINDSroot + "/core/person/v1.0.0")],
        self.schema_json["embargoStatus"] = "minds/core/embargostatus/v1.0.0/1d726b76-b176-47ed-96f0-b4f2e17d5f19",
        self.schema_json["license"] = [{"@id": "licenses/core/information/v1.0.0/7377a480-6066-4c47-9be8-67c586713ed7"}],
#        self.schema_json["activities"] = [{"@id": "minds/core/activity/v1.0.0"+"/"+file} for file in os.listdir(MINDSroot + "/core/activity/v1.0.0")]
        self.schema_json["specimenGroups"] = [{"@id": "minds/core/specimengroups/v1.0.0"+"/"+file} for file in os.listdir(MINDSroot + "/core/specimengroup/v1.0.0")]

## Specify schemas and write out openMINDS metadata
Here we define what key-value pairs shall appear in each JSON schema. (Un)comment according to data set.

In [ ]:
## III. Specify schemas and instances. Fill & create JSONs using the functions/classes specified above.
##### person
#person_schema = {
#    "@type": "",
#    "@id": "",
#    "name": "",
#    "shortName": "",
#}
#
#person_vec = ["Pai, Roopa Kalsank", "Doe, John", "Doe, Jane"]
#
##for person_name in person_vec:
##    person = openMINDSschemaWriter("person", MINDSroot, "/core/person/v1.0.0", person_name, person_schema, BIDSroot)
##    person.fillJSON()
##    person.writeJSON()
#

##### ethics authority
#authority_schema = {
#  "@type": "",
#  "@id": "",
#  "name": ""
#}
#authority_vec = ["Ethics Board of ABC University"]
#
#for authority_name in authority_vec:
#    authority = openMINDSschemaWriter("authority", MINDSroot, "/ethics/authority/v1.0.0", authority_name, authority_schema, BIDSroot)
#    authority.fillJSON()
#    authority.writeJSON()
#
##### ethics approval
#approval_schema = {
#    "@type": "",
#    "@id": "",
#    "name": "",
#    "generatedBy": []
#}
#approval_vec = ["EA/ID/01"]
#
#for approval_name in approval_vec:
#    approval = openMINDSschemaWriter("approval", MINDSroot, "/ethics/approval/v1.0.0", approval_name, approval_schema, BIDSroot)
#    approval.fillJSON()
#    approval.writeJSON()
#
##### method
##1. Depends on how many modalities you include / which processing stages you include.
##2. this script includes acquisition, image processing, creation of TVB-format files & phenotypic/assessment data.
##3. if user edits this, user will also need to edit "activity-methodrange" relationships in the activity schema in Part II.
#
#method_schema = {
#    "@type": "",
#    "@id": "",
#    "name": ""
#}
#
#method_vec = [#acquisition
#              "T1-weighted magnetic resonance imaging (T1w-MRI)",
#              "T2-weighted magnetic resonance imaging (T2w-MRI)",
#              "..."
#             ]
#for i,method_name in enumerate(method_vec):
#    method = openMINDSschemaWriter("method", MINDSroot, "/experiment/method/v1.0.0", method_name, method_schema, BIDSroot,IRI_vec[i])
#    method.fillJSON()
#    method.writeJSON()
#
##### activity
## One *activity* needed for each *method*. Multiple *methods* can combine to create an *activity*. e.g. fMRI (method) + resting state (method) = resting state fMRI (activity)
#
#activity_schema = {
#    "@type": "",
#    "@id": "",
#    "name": "",
#    "approval": [],
#    "authority": [],
#    "methods": [],
#    "preparation":[]
#}
#
#activity_vec = [#acquisition
#                "MRI-T1w",
#                "MRI-T2W",
#                "MRI-T2STAR",
#                "MRI-FLAIR",
#                "resting state fMRI",
#                "DWI",
#                "PET",
#                #processing
#                "DWI-ImageProcessing",
#                "T1-ImageProcessing",
#                "rsfMRI-ImageProcessing"
#                "PET-ImageProcessing",
#
#                #reorganize script/create TVB input files
#                "create cortical surface and region mapping"
#                "compute source space",
#                "compute BEM model & EEG locations",
#                "compute forward solution",
#                "save derivatives accoording to TVB specifications",
#
#                # phenotypic data
#                "PhenotypicandAssessmentData"
#                ]
##vec order important
#
#for activity_name in activity_vec:
#    activity = openMINDSschemaWriter("activity", MINDSroot, "/core/activity/v1.0.0", activity_name, activity_schema, BIDSroot)
#    activity.fillJSON()
#    activity.writeJSON()
#
##### age category
#agecat_schema = {
#  "@type": "",
#  "@id": "",
#  "name": ""
#}
#age_vec = ["22-25","26-30","31-35","36-122"]
#
#for agecat_name in age_vec:
#    agecat = openMINDSschemaWriter("agecategory", MINDSroot, "/core/agecategory/v1.0.0", agecat_name, agecat_schema, #BIDSroot)
#    agecat.fillJSON()
#    agecat.writeJSON()

##### specimen group
specimengroup_schema = {
  "@type": "",
  "@id": "",
  "name": "",
  "subjects": []
}

specimengroup_vec = ["Cognitively Normal (CN)"]

for specimengroup_name in specimengroup_vec:
    specimengroup = openMINDSschemaWriter("specimengroup", MINDSroot, "/core/specimengroup/v1.0.0", specimengroup_name, specimengroup_schema, BIDSroot)
    specimengroup.fillJSON()
    specimengroup.writeJSON()


#### subject
subject_schema = {
  "@type": "",
  "@id": "",
  "name": "",
  "age": "",
  "ageCategory": [],
  "sex": [],
  "species": []
}
    
subject_vec = [sub for sub in participants_tsv["participant_id"].tolist()]

for subject_name in subject_vec:
    subject = openMINDSschemaWriter("subject", MINDSroot, "/experiment/subject/v1.0.0", subject_name, subject_schema, BIDSroot)
    subject.fillJSON()
    subject.writeJSON()

#### loop back and add relevant data to specimen group JSONs
for sg_JSON_file in os.listdir(MINDSroot + "/core/specimengroup/v1.0.0"):
    with open(MINDSroot + "/core/specimengroup/v1.0.0"+"/"+sg_JSON_file,"r") as f:
        sg_JSON_data = json.load(f)
        sg_name = sg_JSON_data["name"]
        if "CN" in sg_name:
            for sub in participants_tsv["participant_id"].tolist():
                sg_JSON_data["subjects"].append({"@id": "minds/experiment/subject/v1.0.0/HCP_sub-" +sub+".json"})
#        elif "MCI" in sg_name:
#            for sub in participants_tsv["participant_id"].tolist():
#                if "MCI" in participants_tsv[participants_tsv["participant_id"]==sub]["Research Group"].item():
#                    sg_JSON_data["subjects"].append({"@id": "minds/experiment/subject/v1.0.0/"+sub+".json"})
#        elif "AD" in sg_name:
#            for sub in participants_tsv["participant_id"].tolist():
#                if participants_tsv[participants_tsv["participant_id"]==sub]["Research Group"].item() == "AD":
#                    sg_JSON_data["subjects"].append({"@id": "minds/experiment/subject/v1.0.0/"+sub+".json"})
    with open(MINDSroot + "/core/specimengroup/v1.0.0"+"/"+sg_JSON_file,"w") as f:
        json.dump(sg_JSON_data, f, indent=4)

#### finally, write the dataset JSON
#Notes/assumptions:
#1. owners: default: first "person"
#2. contributors: all persons
#3. embargoStatus: leave as is for now
#4. license: leave as is for now
#5. description: hardcode or editable? for the moment, hardcode inside class definition. if other schemas also take descriptions, add "description" as function argument.

dataset_schema = {
  "@type": "",
  "@id": "",
  "name": "",
  "description": "",
  "owners": [],
  "contributors": [],
  "embargoStatus": "",
#  "license": [],
#  "activities": [],
  "specimenGroups": []
}

dataset_vec = ["Human Connectome Project Young Adult Structural Connectomes, Functional Connectomes and fMRI time series for connectome analsis and brain network modelling"]

for dataset_name in dataset_vec:
    dataset = openMINDSschemaWriter("dataset", MINDSroot, "/core/dataset/v1.0.0", dataset_name, dataset_schema, BIDSroot)
    dataset.fillJSON()
    dataset.writeJSON()